In [6]:
import pandas as pd # used for DataFrames

In [7]:
# magnetic data xyz file to python lists
clovis_mag = "CLOVIS/clovis_mag.xyz" # magnetic data file path
xyz_mag = open(clovis_mag, "r") # create xyz file object
# separating xyz data columns into lists
line = []
fid = []
time = []
day = []
year = []
latitude = []
longitude = []
radalt = []
totmag = []
resmag = []
diurnal = []
geology = []
resmagCM4 = []
# adds columns of data into corresponding feature list
for row in xyz_mag:
    # Splits rows of data into lists
    row_array = row.split()
    # Adds column of data to corresponding list
    # Casts value to correct data type for later use
    line.append(int(row_array[0]))
    fid.append(int(row_array[1]))
    time.append(int(row_array[2]))
    day.append(int(row_array[3]))
    year.append(int(row_array[4]))
    latitude.append(float(row_array[5]))
    longitude.append(float(row_array[6]))
    radalt.append(float(row_array[7]))
    totmag.append(float(row_array[8]))
    resmag.append(float(row_array[9]))
    diurnal.append(float(row_array[10]))
    geology.append(row_array[11])
    resmagCM4.append(float(row_array[12]))

In [8]:
# DataFrame for magnetic data
mag_DataFrame = pd.DataFrame(zip(line, fid, time, day, year, latitude, longitude, radalt, totmag, resmag, diurnal, geology, resmagCM4), 
            columns=["line", "fid", "time", "day", "year", "latitude", "longitude", "radalt", "totmag", "resmag", "diurnal", "geology", "resmagCM4"])

In [9]:
mag_DataFrame.head(10)

,line,fid,time,day,year,latitude,longitude,radalt,totmag,resmag,diurnal,geology,resmagCM4
0,16,211,115600,103,1976,34.6960,-101.9878,139.9,53303.7,-176.9,-9999.9,TO,107.0
1,16,212,115600,103,1976,34.6960,-101.9878,139.9,53303.7,-176.9,-9999.9,TO,107.0
2,16,213,115600,103,1976,34.6968,-101.9883,137.8,53304.4,-176.8,-9999.9,TO,107.2
3,16,214,115600,103,1976,34.6974,-101.9888,136.2,53306.5,-175.0,-9999.9,TO,109.0
4,16,215,115600,103,1976,34.6981,-101.9893,135.3,53304.8,-177.1,-9999.9,TO,106.8
5,16,216,115600,103,1976,34.6987,-101.9898,135.0,53305.2,-177.0,-9999.9,TO,107.0
6,16,217,115600,103,1976,34.6992,-101.9903,134.4,53306.6,-175.9,-9999.9,TO,108.1
7,16,218,115600,103,1976,34.6997,-101.9908,136.9,53304.8,-177.8,-9999.9,TO,106.2
8,16,219,115600,103,1976,34.7004,-101.9913,138.7,53306.7,-176.2,-9999.9,TO,107.7
9,16,220,115600,103,1976,34.7012,-101.9918,138.7,53306.1,-177.3,-9999.9,TO,106.8


In [10]:
# radiometric xyz data to python lists
clovis_rad = "CLOVIS/clovis_rad.xyz" # radiometric data file path
xyz_rad = open(clovis_rad, "r") # create xyz file object
# separating xyz data columns into lists
line = []
fid = []
time = []
day = []
year = []
latitude = []
longitude = []
radalt = []
resmag = []
geology = []
qual = []
app_K = []
app_U = []
app_Th = []
U_Th_ratio = []
U_K_ratio = []
Th_K_ratio = []
total_count = []
atmos_BI214 = []
air_temp = []
air_press = []
# adds columns of data into corresponding feature list

['1',
 '1',
 '171900',
 '102',
 '1976',
 '34.0081',
 '-104.0169',
 '141.1',
 '-266.0',
 '-99',
 '-9999',
 '140.0',
 '9.0',
 '27.0',
 '-999.9',
 '-999.9',
 '-999.9',
 '3614.0',
 '601.0',
 '20.8',
 '651.3']